In [16]:
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
# Veri setinin bulunduğu klasörü belirtin
data_dir = 'lfw-deepfunneled/'

# Görüntüleri ve etiketleri depolamak için boş listeler oluşturun
images = []
labels = []


In [18]:
# Tüm klasörleri döngüye alarak görüntüleri yükleyin ve etiketleri oluşturun
for person_name in os.listdir(data_dir):
    person_dir = os.path.join(data_dir, person_name)
    if os.path.isdir(person_dir):
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(image)
            labels.append(person_name)

In [19]:
class_names = np.unique(labels)
class_indices = {class_name: i for i, class_name in enumerate(class_names)}


In [20]:
# Etiketleri sayısal forma dönüştürme
labels = [class_indices[label] for label in labels]

In [21]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(250, 250, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5749, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 248, 248, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 124, 124, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 61, 61, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 59, 59, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 29, 29, 128)     

In [23]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
331/331 [==============================] - 17s 50ms/step - loss: 8.8277 - accuracy: 0.0331 - val_loss: 8.1572 - val_accuracy: 0.0427
Epoch 2/10
331/331 [==============================] - 25s 75ms/step - loss: 7.8726 - accuracy: 0.0400 - val_loss: 8.4511 - val_accuracy: 0.0442
Epoch 3/10
331/331 [==============================] - 26s 79ms/step - loss: 7.5031 - accuracy: 0.0439 - val_loss: 8.8204 - val_accuracy: 0.0468
Epoch 4/10
331/331 [==============================] - 26s 78ms/step - loss: 6.6823 - accuracy: 0.0488 - val_loss: 9.3245 - val_accuracy: 0.0412
Epoch 5/10
331/331 [==============================] - 31s 94ms/step - loss: 4.7840 - accuracy: 0.1678 - val_loss: 12.4268 - val_accuracy: 0.0268
Epoch 6/10
331/331 [==============================] - 31s 94ms/step - loss: 2.1047 - accuracy: 0.5691 - val_loss: 18.9380 - val_accuracy: 0.0185
Epoch 7/10
331/331 [==============================] - 32s 98ms/step - loss: 0.6775 - accuracy: 0.8503 - val_loss: 23.7580 - val_accura

In [26]:
from sklearn.metrics import f1_score

# Tahminleri al
y_pred_train = model.predict(X_train)
y_pred_train = np.argmax(y_pred_train, axis=1)

# Metrikleri hesapla
train_accuracy = accuracy_score(y_train, y_pred_train)
f1_micro = f1_score(y_train, y_pred_train, average='micro')
f1_macro = f1_score(y_train, y_pred_train, average='macro')
precision = precision_score(y_train, y_pred_train, average='weighted', zero_division=0)
recall = recall_score(y_train, y_pred_train, average='weighted', zero_division=0)

331/331 [==============================] - 27s 83ms/step


In [27]:
print(f'Eğitim Doğruluk: {train_accuracy}')
print(f'F1 Mikro: {f1_micro}')
print(f'F1 Makro: {f1_macro}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

Eğitim Doğruluk: 0.9829964103532968
F1 Mikro: 0.9829964103532968
F1 Makro: 0.9834884952995897
Precision: 0.9832492279288791
Recall: 0.9829964103532968


In [31]:
fscore = 2 * (precision * recall) / (precision + recall)
print(f'F1 Score: {fscore}')

F1 Score: 0.9831228028875932


In [15]:
#Bu model, evrişimli sinir ağı (CNN) kullanılarak bir görüntü sınıflandırma modelidir. Modelin katmanları şu şekilde oluşturuldu:

#1. **Conv2D Katmanları:** Giriş olarak 250x250 piksel boyutlarındaki renkli (RGB) görüntüler kullanılmıştır. İlk Conv2D katmanı, 32 filtre ile 3x3 boyutunda evrişim gerçekleştirir. Ardından, 2x2 boyutunda maksimum havuzlama (MaxPooling) katmanı ile boyutu küçültülür.

#2. **İkinci Conv2D Katmanları:** Ardından bir ikinci Conv2D katmanı gelir, bu sefer 64 filtre ile çalışır ve yine 3x3 boyutunda evrişim yapar. Ardından tekrar bir 2x2 MaxPooling katmanı kullanılır.

#3. **Üçüncü Conv2D Katmanları:** Bir üçüncü Conv2D katmanı eklenir, bu sefer 128 filtre ile çalışır ve yine 3x3 boyutunda evrişim yapar. Ardından bir kez daha 2x2 boyutunda MaxPooling katmanı gelir.

#4. **Flatten Katmanı:** Evrişim katmanlarından sonra, Flatten katmanı kullanılarak düzleştirme işlemi yapılır. Bu, evrişim katmanlarının çıktılarını düzleştirerek tek bir vektör haline getirir.

#5. **Dense (Tam Bağlı) Katmanlar:** Ardından, iki tam bağlı (dense) katman gelir. İlk tam bağlı katman 512 nörona sahiptir ve aktivasyon fonksiyonu olarak ReLU kullanır. İkinci tam bağlı katman, sınıf sayısı olan 5749 nörona sahiptir ve softmax aktivasyon fonksiyonu kullanır. Bu, modelin birden çok sınıfı sınıflandırabilmesini sağlar.

#Modelin çıktı katmanındaki nöron sayısı, sınıf sayısına eşittir ve softmax aktivasyon fonksiyonu kullanılarak her bir sınıfa ait olasılık değerleri elde edilir. Bu, çoklu sınıf sınıflandırma problemleri için yaygın bir yapıdır.

#Modelin derlenmesi (compile) sırasında "adam" optimizasyon algoritması ve "sparse_categorical_crossentropy" kayıp fonksiyonu kullanılmıştır. Modelin başarı ölçümü olarak "accuracy" (doğruluk) kullanılmıştır.
